Hidden Markov Model:
实验的比较：

（1）原文(Isabel Kreis):        
stimuli:呈现垂直光栅（等同于凝视点）后，出现左或右两个朝向的gabor    
被试resp：预测下一次会出现左还是右的gabor  
feedback:在下一个试次中以stimulus呈现

（2）windows-animals实验  
stimulus--不同颜色的window  
被试resp：预测背后的动物  
feedback：直接反馈对或错

如果让两个实验一致，可以理解为在w-a实验中，呈现的stimulus为window-animal 的联结，被试resp：预测下一次会出现什么联结（association）。依照原文的实验，把stimuli也就是本实验中的asso分为两类:

1-----黄色--猫/蓝色--狗-------原文中的left朝向

2-----蓝色--猫/黄色--狗---------------right朝向

-------------------------------------这是一条分界线------------------------------------

这部分是导入数据，但是应该把原先的u_t换成实际的feedback（）和选择

In [ ]:
#  This program run nback model
import numpy as np
import stan
import pickle
from time import time

In [ ]:
action = np.loadtxt("demo_files/action.txt", dtype=int)
rewards = np.loadtxt("demo_files/rewards.txt", dtype=int)
T = 320
mini = np.finfo(float).eps
# pack the data(但这里的data应该转变)
data_dict = {
    'T'  : T,
    'mini':mini,
    'action' :list(action),#这里应该把u_t/resp改为reward/action,其中action对应的就是category 的联结，reward即±1
    'resp':list(rewards)
}


写stan model的思路

先写model部分
参数（需要被采样的）：
      γ（gamma）：transition probability（0~1）
      c: reward sensitivity （0.5~1）
      d：punishment sensitivity（0.5~1）

写生成关于action的预测：（update function）



prior of P(St|St-1):
P(St|St-1) = |1-gamma    gamma|
             |gamma    1-gamma|
             一共只有两个state

In [ ]:
'''
P(St|St-1):follow transition matrix(每个trial之间独立，相当于2states-2actions)
            S1          s2
    S1      1-gamma     gamma
    S2      gamma       1-gamma
    
Ps[1] = (1-gamma) * Ps[1] + gamma * Ps[2]
Ps[2] = 1-Ps[1]
'''

有了obs之后产生action

In [ ]:
# action selection based on the State probability 
'''choice[t] ~ categorical(Ps)'''

但以上是latent states ，被试还会因为对reward/punishment的敏感程度不同（c/d）获得观察结果的概率（可以理解为s---（measurement noise）-----x？）
the probability of actually observing this outcome

In [ ]:
'''
P(Ot|Ot)/0.5:
            S1          S2
    O1      c           1-c
    O2      1-d         d
P0[1] = 0.5 * (c*St[1] +(1-c)*St[2])
PO[2] = 0.5 * ((1-d)*St[1] + d*St[2])  

'''

在获得outcome之后，St的posterior prob就会根据P（St）和outcome进行合并更新
公式为：P(St) = P(Ot|St)*P(St)/((ΣSt P(Ot|St)*P(St)))
    
    
    P1 + P2 = 1

合并以上内容

In [ ]:
hmm='''
data {
    real mini; // to avoid p = 0 or 1-p=0,
  
    int<lower=0> T; // 试次数量
  
    array[T] int<lower=-1, upper=1> reward; // 
    array[T] int<lower=1, upper=2> action; // 

transformed data{
  vector[2] init_Ps; // initial prob/belief of the two states
  init_Ps = rep_vector(0.5, 2);
}

parameters {
    real <lower=0,upper=1> gamma_pr; 
    real <lower=0.5,upper=1> c_pr; 
    real <lower=0.5,upper=1> d_pr;
}

transformed parameters{
    real gamma;
    real c;
    real d;
    vector[2] Ps;  // prob of the states, 1 - yellow-cat(blue-dog), 2 - yellow-dog(blue-cat)
    real P_O_S1;   // p(O|S1) - O = {A,R} given yellow-cat
    real P_O_S2;   // p(O|S2) - O = {A,R} given yellow-dog
  
    real gamma = Phi_approx(gamma_pr);//将先验参数进行转换和归一化,以确保其满足所需的范围和约束条件。将输入值通过逆标准正态累积分布函数进行转换。范围（0-1）
    real c = Phi_approx(c_pr) * 0.5 + 0.5;//范围(0.5-1)
    real d = Phi_approx(d_pr) * 0.5 + 0.5;//范围(0.5-1)
    
    //*******************compute ******************************
    for (t in 1:T)  {  
    // 使用 transition matrix 进行State update
    // S[t-1] -- S[t],obs之前
    if (t == 1) {
      Ps = init_Ps;
    } 
    else {
      Ps[1] = Ps[1] * (1 - gamma) + Ps[2] * gamma;
      Ps[2] = 1 - Ps[1];
    }

    // action selection based on the State probability 
    choice[t] ~ categorical(Ps);
      
    // renew of emission prob: p(O|S1) p(O|S2); O 为 A(ction) 和 R(eward)的组合
    // --> the probability of actually observing this outcome
    if (reward[t] == 1) {
      P_O_S1 = 0.5 * ((choice[t] == 1) ? c : (1 - c));
      P_O_S2 = 0.5 * ((choice[t] == 2) ? c : (1 - c));
    } else if (reward[t] == -1) {
      P_O_S1 = 0.5 * ((choice[t] == 1) ? (1 - d) : d);
      P_O_S2 = 0.5 * ((choice[t] == 2) ? (1 - d) : d);
    }

    // State belief update 
    Ps[1] = (P_O_S1 * Ps[1]) / (P_O_S1 * Ps[1] + P_O_S2 * Ps[2]);
    Ps[2] = 1 - Ps[1];
    } 
  }
}
}

model {  
  // prior of  parameters
  gamma_pr ~ normal(0, 1);
  c_pr ~ normal(0.5, 1);
  d_pr ~ normal(0.5, 1);
  
  for (t in 1:T){
    action[t] ~ bernoulli(choice[t]*0.999+mini);
  }
  
}
'''

In [ ]:
sm = stan.build(hmm, data=data_dict)  # specify the model file

In [ ]:

nChain = 4  # How Many Chains, depend on the # of cores of your computer. 4 seems good
nWarmup = 10000  # How Many Burn-in Samples?
nIter = 10000  # How Many Recorded Samples, including nWarmup samples
save = True # whether save the fitting result or not
watch_list = ['gamma_pr', 'c_pr', 'd_pr']